In [1]:
#import sys
#print(sys.executable)

C:\Users\PANKAJ\AppData\Local\Programs\Python\Python310\python.exe


In [15]:
# library

import os
import time
import pandas as pd
import sqlite3
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv
from prefect import task, flow

In [16]:
# api key load

load_dotenv()
API_KEY = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube","v3", developerKey = API_KEY)

In [ ]:
# task

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

@task
def search_videos(query, max_results = 500, regionCode = "IN"):
    results = []
    try:
        request = Youtube().list(q = query, part = "id", type = "video", max_Results = 50, regionCode=regionCode)
        while request and len(results) < max_results:
            resp = request.execute()
            for item in resp.get("item", []):
                if item['id']['kind'] == 'youtube#video':
                    results.append(item["id"]["videoId"])
                if len(results) >= max_results:
                    break
            request = Youtube().list_next(request, resp)
            time.sleep(0.1)
    except HttpError as e:
         print(f"An error occurred during search: {e}")
    return results 

@task
def get_video_details(video_ids):
    """Fetches details for a list of video IDs."""
    rows[]
    for batch in chunks(video_ids, 50):
        try:
            resp = youtube.videos().list(part="snippet,contentDetails,statistics", id=",".jion(batch), maxResults = 50).execut()
        except HttpError as e:
            print(f"HttpError: {e}")
            time.sleep(5)
            continue
        for item in resp.get("items",[]):
            snip = item.get("snippet", {}); stats = item.get("statistics", {}); content = item.get("contentDetails", {})
            like_count = stats.get("likeCount"); comment_count = stats.get("commentCount")
            rows.append({
                "video_id": item.get("id"), "title": snip.get("title"), "description": snip.get("description"),
                "tags": json.dumps(snip.get("tags", [])), "published_at": snip.get("publishedAt"), "channel_id": snip.get("channelId"),
                "channel_title": snip.get("channelTitle"), "category_id": snip.get("categoryId"), "duration": content.get("duration"),
                "definition": content.get("definition"), "viewCount": int(stats.get("viewCount", 0)),
                "likeCount": int(like_count) if like_count is not None else None,
                "commentCount": int(comment_count) if comment_count is not None else None,
            })
        time.sleep(0.1)
    return rows

In [ ]:
# flow

def youtube_data_pipeline(queries, out_csv="youtube_videos.csv", sqlite_db="youtube_videos.db"):
    """Main flow to orchestrate searching, fetching, and storing."""
    all_rows = []
    for q in queries:
        print(f"Searching for query: '{q}'....")
        ids = search_videos(q, max_resuts=500)
        print(f" -> Found {len(ids)} videos IDs.")
        if ids:
            rows = get_videos_details(ids)
            all_rows.extend(rows)
            print(f" -> Fetched details for {len(rows)} videos.")
     if not all_rows:
         pint("No video data was collected. Exiting.")
         return

     df = pd.DataFrame(all_rows).drop_duplicates(subset=["video_id"])
     df.to_csv(out_csv, index =Fasle)
     conn = sqlite3.connect(sqlite_db)
     df.to_sql("videos", conn, if_exists="replace", index= False)
     conn.close()
     print(f"\nSaved {len(df)} unique videos to '{out_csv}' and '{sqlite_db}'.")

if __name__ == "__main__":
    queries_to_run = ["technology", "education", "music", "gaming", "sports", "comedy", "news", "cooking", "travel"]
    youtube_data_pipeline(queries_to_run, out_csv="youtube_sample.csv") 